In [13]:
import torch
from torch.utils.data import DataLoader
import os
import pandas as pd
from work.utils.dataset import PandasDataset, RGB2Fusion
import albumentations
from work.utils.models import EfficientNet
from work.utils.metrics import evaluation
from sklearn.metrics import confusion_matrix

In [14]:
output_dimensions = 5
data_dir = '../../dataset'
images_dir = os.path.join(data_dir, 'tiles')
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
df_test = pd.read_csv(f"../data/test.csv")
backbone_model = 'efficientnet-b0'
pretrained_model = {
    backbone_model: '/home/woshington/Projects/Doutorado/work/efficientnet-b0-08094119.pth'
}

In [15]:
print("Cuda", device)

Cuda cuda


In [16]:
val_transform =albumentations.Compose([
    RGB2Fusion(mode="sum", space_colors=["rgb", "xyz", "lab"]),
])

In [17]:
dataloader = DataLoader(
    PandasDataset(images_dir, df_test, transforms=val_transform),
    batch_size=2,
    shuffle=False,
)

In [18]:
model = EfficientNet(
    backbone=backbone_model,
    output_dimensions=output_dimensions,
    pre_trained_model=pretrained_model
)
model.to(device)
model.load_state_dict(
    torch.load(
        "models/fusion-sum.pth",
        weights_only=True
    )
)

response_0 = evaluation(model, dataloader, device)
print("without treatment", response_0[0])


Loaded pretrained weights for efficientnet-b0


100%|██████████| 796/796 [34:12<00:00,  2.58s/it]


without treatment {'val_acc': {'mean': 60.055276489257814, 'std': 1.2090192720321227, 'ci_5': 58.16583037376404, 'ci_95': 62.000626027584076}, 'val_kappa': {'mean': 0.8200664899763548, 'std': 0.012367270783631834, 'ci_5': 0.7992301494415793, 'ci_95': 0.8402981682733633}, 'val_f1': {'mean': 0.5398267081975937, 'std': 0.01271082250939262, 'ci_5': 0.5191817820072174, 'ci_95': 0.5601608514785766}, 'val_recall': {'mean': 0.5390451556444168, 'std': 0.012704715731874408, 'ci_5': 0.5181984335184098, 'ci_95': 0.5593686908483505}, 'val_precision': {'mean': 0.5450801727771759, 'std': 0.012868361050821592, 'ci_5': 0.5238999515771866, 'ci_95': 0.5658688336610794}}


In [19]:
matriz = confusion_matrix(response_0[1][1], response_0[1][0])

In [20]:
print(matriz)

[[382  27  13   2   8   2]
 [ 73 244  70  11   2   0]
 [  9  54  82  44  11   1]
 [  5  11  33  75  47  15]
 [  9  12  16  27  79  44]
 [  5   6   6  17  56  94]]
